# Intro

As a first test for DKT elements, in this notebook we find minimal configurations (of the vertical displacements $u$) for the energy

$$ I_{K i}(w) = \frac{1}{2} \int_\omega |D^2 w|^2 \mathrm{d}x - \int_\omega f_3 w \mathrm{d}x$$

under homogeneous Dirichlet boundary conditions. This means solving the linear problem

$$ (D^2w, D^2v) = (f_3,v),$$

for all $v \in W$. We implement a the non-conforming discretisation

$$ (\nabla \nabla_h w_h, \nabla \nabla_h v_h) = (f_3, v_h), $$

for all $v_h \in W_{h,D} := ...$

The crux of the matter is the [discrete gradient operator](discrete_gradient.ipynb) $\nabla_h$. Recall that $\nabla_h : W_h \rightarrow \Theta_h$ is defined as...

# DKT discrete gradient and UFL

Our naive implementation of `compute_dkt_gradient()` for Functions won't work for `Arguments`: we would need to instruct UFL how to generate UFC code for the form and integrals. However, it is not clear that this is possible at all: UFL forms are element-agnostic, meaning that they have no knowledge of the basis of shape functions. In particular, they are meant to act on *one* generic basis function at a time, so that it seems impossible to define a discrete operator which acts on all basis functions of one cell.

In [ ]:
from dolfin import *
import nbimporter
from discrete_gradient import dkt_gradient_operator, compute_dkt_gradient

W = FunctionSpace(UnitSquareMesh(1,1), 'DKT', 3)
T = VectorFunctionSpace(W.mesh(), 'Lagrange', 2, dim=2)
w = TrialFunction(W)
v = TestFunction(W)

Dh = dkt_gradient_operator(W, T)

class Boundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

bc = DirichletBC(W, Constant(0.), Boundary())

def gradh(u):
    # FIXME: won't work for Arguments!
    return compute_dkt_gradient(W, T, u)

a = inner(nabla_grad(gradh(w)), nabla_grad(gradh(v)))*dx
L = inner(Constant(-9.8), v)*dx

u = Function(W)
solve(a == L, u)

# Manual assembly of the stiffness matrix

The alternative to extending UFL is to build the system matrix ourselves. Here is one possible way of doing this which reuses what FFC already provides. Let $T \in \mathcal{T}_h$ be one cell in the triangulation. We want to
assemble the **local tensor** for the bilinear form over the DKT
element, $P^{\text{red}}_3 (T)$:

$$ (\nabla \nabla_h \phi_j, \nabla \nabla_h \phi_i)
  = (f_3, \phi_i), \text{ for all } \phi_i, \phi_j \in P^{\text{red}}_3 (T) . $$

Let $\theta_l, \theta_k \in P_2^2 (T)$ and let
$$ A^{(2)}_{l  k} = (\nabla \theta_k, \nabla \theta_l), \quad k, l \in
   [12] $$
be the **local tensor** for the $P_2^2$ element over $T$. Recall that
$M_T = (m_{p  q}) \in \mathbb{R}^{12 \times 9}$ is the matrix for the
local discrete gradient. Then, for every $\phi_i, i \in [9]$:

$$ \nabla_h \phi_i = M_T  \left(\begin{array}{c}
     0\\
     \vdots\\
     1\\
     \vdots\\
     0
   \end{array}\right) = (M_T)_{l  i} \theta_l = m_{l  i}
   \theta_l, $$
and analogously $ \nabla_h \phi_j = m_{k  j} \theta_k . $

Substituting back in (\ref{eq:local-tensor-lki}) we obtain

$$ A_{i  j} = (\nabla \nabla_h \phi_j, \nabla \nabla_h \phi_i) = m_{k
    j} m_{l  i}  (\nabla \theta_k, \nabla \theta_l) = m_{k
    j} m_{l  i} A_{l  k}^{(2)} . $$

To summarize:

1. Compile with `ffc` the form inner$(\nabla u, \nabla v)$\*dx for `VectorFunctionSpace<P2>`. This provides us with the local cell tensor $A^{(2)}$ in `cell_integral::tabulate_tensor`.
2. Compile with `ffc` the form inner$(u, v)$\*dx for `FunctionSpace<DKT>`. The dofmaps, etc. should be ok. This provides us with the **wrong** local cell tensor. In order to compute the right one, $A^{(1)}$:
    1. Compute the local matrix for $\nabla_h$ over simplex $t$: $M_t = (m_{p q})$.
    2. Compute 
    $$A_{i j}^{(1)} = m_{k j} m_{l i} A_{l k}^{(2)} = (M_{: i})^T A^{(2)} M_{: j}$$
    3. Do this inside `cell_integral::tabulate_tensor` for the form.
3. Let dolfin assemble the matrix

## The form in $P_2^2$

We first generate the code for $(\nabla u, \nabla v)$ ('compile' in ffc terms):

In [ ]:
from ffc import compile_form
from dolfin import *

V = VectorFunctionSpace(UnitSquareMesh(1,1), "Lagrange", 2, dim=2)
u = TrialFunction(V)
v = TestFunction(V)
a = inner(nabla_grad(u), nabla_grad(v))*dx
with open("grad-grad-vecP2.h", "wt") as fd:
    code = compile_form(a)
    fd.write(code[0])

[This guide](http://hplgit.github.io/fenics-mixed/doc/pub/sphinx-cbc/._part0003_fenics-mixed.html) offers info on how to compile C++ extensions and make them available as python modules, but this is not enough for forms and elements, since dolfin has to be aware of them. So the following code won't be enough:

```python
grad_grad_module = compile_extension_module(code=code[0], source_directory='.',
                                            sources=[], include_dirs=["."])
```

Instead, we mimic what dolfin does when we call `assemble` for a form and use `jit`:

In [ ]:
from dolfin.fem.form import Form

In [ ]:
# TODO... HERE...

## The form in DKT

In [ ]:
W = FunctionSpace(V.mesh(), "Hermite", 3)
u = TrialFunction(V)
v = TestFunction(V)
a = inner(u, v)*dx
with open("u-v-her.h", "wt") as fd:
    out = compile_form(a)
    fd.write(out[0])

# ufc_utils

One can use the templates in `ffc.backends.ufc` to generate the classes inheriting from `ufc::form` and `uf::cell_integral`.

In [ ]:
from ffc.backends.ufc import form as uf

impl = dict(classname = 'linear_kirchhoff',
            constructor = '// Do nothing',
            constructor_arguments = '',
            create_cell_integral = '',
            create_coordinate_dofmap = '',
            create_coordinate_finite_element = '',
            create_coordinate_mapping = '',
            create_custom_integral = '',
            create_cutcell_integral = '',
            create_default_cell_integral = '',
            create_default_custom_integral = '',
            create_default_cutcell_integral = '',
            create_default_exterior_facet_integral = '',
            create_default_interface_integral = '',
            create_default_interior_facet_integral = '',
            create_default_overlap_integral = '',
            create_default_vertex_integral = '',
            create_dofmap = '',
            create_exterior_facet_integral = '',
            create_finite_element = '',
            create_interface_integral = 'return 0;',
            create_interior_facet_integral = 'return 0;',
            create_overlap_integral = 'return 0;',
            create_vertex_integral = '',
            destructor = '// Do nothing',
            has_cell_integrals = '',
            has_custom_integrals = '',
            has_cutcell_integrals = '',
            has_exterior_facet_integrals = '',
            has_interface_integrals = '',
            has_interior_facet_integrals = '',
            has_overlap_integrals = '',
            has_vertex_integrals = '',
            initializer_list = '',
            max_cell_subdomain_id = '',
            max_custom_subdomain_id = '',
            max_cutcell_subdomain_id = '',
            max_exterior_facet_subdomain_id = '',
            max_interface_subdomain_id = '',
            max_interior_facet_subdomain_id = '',
            max_overlap_subdomain_id = '',
            max_vertex_subdomain_id = '',
            members = '',
            num_coefficients = 'return 0;',
            original_coefficient_position = 'return 0;',
            rank = '',
            signature = 'return "Linear Kirchhoff plate";')
print(uf.form_header % impl)
print(uf.form_implementation % impl)

implementation = {}

implementation["signature"] = 'return "my form"'
implementation["rank"] = "return 2;" 
implementation["num_coefficients"] = "return 0;" 
implementation["num_cell_domains"] = "return 3;"
implementation["num_interior_facet_domains"] = "return 1;" 
implementation["num_exterior_facet_domains"] = "return 0;" 

implementation["create_finite_element"] = """
switch (i)
{
case 0:
  return new my_finite_element_0();
case 1:
  return new my_finite_element_1();
default:
return 0; }"""
implementation["create_dofmap"] = """ switch (i)
{
case 0:
  return new my_dofmap_0();
case 1:
  return new my_dofmap_1();
default:
return 0; }"""
implementation["create_cell_integral"] = """ switch (i)
{
case 0:
  return new my_cell_integral_0();
case 1:
  return new my_cell_integral_1();
case 2:
  return new my_cell_integral_2();
default:
return 0; }"""
implementation["create_exterior_facet_integral"] = "return new my_exterior_facet_integral();"